# Tournois Binomiaux

In [ ]:
import graphviz as gv
import math as m
import os as file

class TournoiBinomial:
    def __init__(self,k):
        self.pere = None # TournoiBinomial
        self.fils = FileBinomial() # FileBinomial comporant k fils
        self.pred = None # TournoiBinomial frere gauche
        self.succ = None # TournoiBinomial frere droit
        self.cle = k # clé de TournoiBinomial
        self.degre = 0 # degré de TournoiBinomial
        
    def EstRacine(T):
        if(T.pere is None):
            return True;
        else:
            return False

    def EstVide(T):
        if (T is None):
            return True
        else:
            return False
        
    def Degre(T):
        return T.degre
    
    def Union2Tid (T1,T2):
        # TournoiB ∗ TournoiB −> TournoiB Renvoie l’union de 2 tournois de meme taille
        if(T1.cle < T2.cle):
            T1.fils.arbres.append(T2)
            T1.degre = T1.Degre() + 1
            return T1
        else:
            T2.fils.arbres.append(T1)
            T2.degre = T2.Degre() + 1
            return T2
        
    def Decapite(T):
        # TournoiB −> FileB Renvoie la file binomiale obtenue en supprimant la racine du tournoi 
        # T_k −> <T_{k−1},T_{k−2}, ... ,T_1,T_0 >
        F = FileBinomial()
        F.arbres.extend(T.fils)
        return F
    
    def File(T):
        # TournoiB −> FileB Renvoie la file binomiale reduite au tournoi
        # T_k −> <T_k >
        F = FileBinomial()
        F.arbres.append(T)
        return F
    
    def to_graph(self,G,i):
        for T in self.fils.arbres:
            # racine de T
            G.node(str(i),str(T.cle),shape='circle')
            if(i > 0 and T.fils.arbres):
                # double chaînage entre les fils
                G.edge(str(i-1),str(i), constraint='false')
                G.edge(str(i),str(i-1), constraint='false')
                # chaînage de la racine vers les fils
                G.edge('racine'+str(self.cle), str(i))
                G.edge('racine'+str(self.cle), str(i-1))
            elif(i > 0 and not T.fils.arbres):
                # double chaînage entre les fils
                G.edge(str(i-1),str(i))
            else:
                # racine du pere
                G.node('racine'+str(self.cle), str(self.cle), shape='doublecircle')
            i = i + 1
            T.to_graph(G,i)
        return G
    
    # Permet d'afficher l'arbre
    def plot(self):
        G = gv.Digraph(strict=True)
        return self.to_graph(G,0)

# Files Binomiales

In [ ]:
class FileBinomial:
    def __init__(self):
        self.arbres = [] # Liste de tournoi binomial
        
    def EstVide(F):
        if F is None:
            return True
        elif not F.arbres:
            return True
        else:
            return False
        
    def MinDeg(F):
        # FileB −> TournoiB Renvoie le tournoi de degre minimal dans la file
        T = TournoiBinomial(0)
        for a in F.arbres:
            if(T.cle == 0):
                T = a
            else:
                if(TournoiBinomial.Degre(a) < TournoiBinomial.Degre(T)):
                    T = a
        return T
    
    def Reste(F):
        # FileB −> FileB Renvoie la file privee de son tournoi de degre minimal
        if(FileBinomial.MinDeg(F) in F.arbres):
            F.arbres.remove(FileBinomial.MinDeg(F))
        return F
    
    def AjoutMin(T, F):
        # Hypothese : le tournoi est de degre inferieur au MinDeg de la file
        # Tournoi ∗ FileB −> FileB
        # Renvoie la file obtenue en ajoutant le tournoi comme tournoi de degre minimal de la file initiale
        F.arbres.append(T)
        return F
    
    def UnionFile (F1 , F2):
        # FileB ∗ FileB −> FileB, Renvoie la file binomiale union des deux files F1 et F2
        return FileBinomial.UFret(F1 , F2 , None)
    
    def UFret(F1 , F2 , T):
        # FileB ∗ FileB ∗ TournoiB−> FileB, Renvoie la file binomiale union de deux files et d’un tournoi
        if TournoiBinomial.EstVide(T): # pas de tournoi en retenue
            if FileBinomial.EstVide(F1):
                return F2
            if FileBinomial.EstVide(F2):
                return F1
            T1 = FileBinomial.MinDeg(F1)
            T2 = FileBinomial.MinDeg(F2)
            if TournoiBinomial.Degre(T1) < TournoiBinomial.Degre(T2):
                return FileBinomial.AjoutMin(T1 , FileBinomial.UnionFile (FileBinomial.Reste(F1), F2))
            if TournoiBinomial.Degre(T2) < TournoiBinomial.Degre(T1):
                return FileBinomial.AjoutMin(T2 , FileBinomial.UnionFile (FileBinomial.Reste(F2), F1))
            if TournoiBinomial.Degre(T1) == TournoiBinomial.Degre(T2):
                return FileBinomial.UFret(FileBinomial.Reste(F1), FileBinomial.Reste(F2), TournoiBinomial.Union2Tid (T1 ,T2))
        else: # T tournoi en retenue
            if FileBinomial.EstVide(F1):
                return FileBinomial.UnionFile (TournoiBinomial.File(T), F2)
            if FileBinomial.EstVide(F2):
                return FileBinomial.UnionFile (TournoiBinomial.File(T), F1)
            T1 = FileBinomial.MinDeg(F1)
            T2 = FileBinomial.MinDeg(F2)
            if TournoiBinomial.Degre(T) < TournoiBinomial.Degre(T1) and TournoiBinomial.Degre(T) < TournoiBinomial.Degre(T2):
                return FileBinomial.AjoutMin(T, FileBinomial.UnionFile (F1 , F2))
            if TournoiBinomial.Degre(T) == TournoiBinomial.Degre(T1) and TournoiBinomial.Degre(T) == TournoiBinomial.Degre(T2):
                return FileBinomial.AjoutMin(T, FileBinomial.UFret(Reste(F1), FileBinomial.Reste(F2), TournoiBinomial.Union2Tid (T1 , T2)))
            if TournoiBinomial.Degre(T) == TournoiBinomial.Degre(T1) and TournoiBinomial.Degre(T) < TournoiBinomial.Degre(T2):
                return FileBinomial.UFret(FileBinomial.Reste(F1), F2 , TournoiBinomial.Union2Tid (T1 , T))
            if TournoiBinomial.Degre(T) == TournoiBinomial.Degre(T2) and TournoiBinomial.Degre(T) < TournoiBinomial.Degre(T1):
                return FileBinomial.UFret(FileBinomial.Reste(F2), F1 , TournoiBinomial.Union2Tid (T2 , T))
            
    def Ajout(cle, F):
        # cle -> FileB, Renvoi la file binomial union de deux files
        T = TournoiBinomial(cle)
        FT = FileBinomial()
        FT.arbres.append(T)
        return FileBinomial.UnionFile(FT,F)
    
    def ConsIter(F):
        # Liste -> FileB, Renvoi la file binomiale en la construisant à partir d'une file
        return None
    
    def minT(F):
        # FileB -> TournoiB, Renvoi le tournois binomial avec la plus petite racine 
        T = None
        for a in F.arbres:
            if(T is None):
                T = a
            else:
                if(a.cle < T.cle):
                    T = a
        return T
    
    def SuppMin(F):
        # FileB −> FileB, Renvoi la file binomiale en supprimant la plus petite racine
        T = minT(F)
        F = Decapite(T)
        return F

# Test Union

In [ ]:
B0_1 = TournoiBinomial(1)
B0_2 = TournoiBinomial(3)
B0_3 = TournoiBinomial(7)
B0_4 = TournoiBinomial(9)
B1_1 = TournoiBinomial.Union2Tid(B0_1,B0_2)

print("Test sans retenue :")
F1 = FileBinomial()
F2 = FileBinomial()
F1.arbres.append(B0_4)
F2.arbres.append(B1_1)
F3 = FileBinomial.UnionFile(F1, F2)
for a in F3.arbres:
    print("LVL0 : " + str(a.cle))

print("Test retenue :")
F4 = FileBinomial()
F4.arbres.append(B0_3)
F5 = FileBinomial.UnionFile(F3, F4)
for a in F5.arbres:
    print("LVL0 : " + str(a.cle))
    for ssa in a.fils.arbres:
        print("LVL1 : " + str(ssa.cle))

# Test Ajout

In [ ]:
B0_1 = TournoiBinomial(1)
B0_2 = TournoiBinomial(3)
B0_3 = TournoiBinomial(7)
B1_1 = TournoiBinomial.Union2Tid(B0_1,B0_2)

F1 = FileBinomial()
F1.arbres.append(B0_3)
F1.arbres.append(B1_1)
F1 = FileBinomial.Ajout(2,F1)
for a in F1.arbres:
    print("LVL0 : " + str(a.cle))
    for ssa in a.fils.arbres:
        print("LVL1 : " + str(ssa.cle))
        for sssa in ssa.fils.arbres:
            print("LVL2 : " + str(sssa.cle))

# Test Graphique

In [ ]:
B0_1 = TournoiBinomial(1)
B0_2 = TournoiBinomial(3)
B0_3 = TournoiBinomial(7)
B0_4 = TournoiBinomial(9)
B1_1 = TournoiBinomial.Union2Tid(B0_1,B0_2) 
B1_2 = TournoiBinomial.Union2Tid(B0_3,B0_4)
B2_1 = TournoiBinomial.Union2Tid(B1_1,B1_2)

B0_5 = TournoiBinomial(2)
B0_6 = TournoiBinomial(4)
B0_7 = TournoiBinomial(6)
B0_8 = TournoiBinomial(8)
B1_3 = TournoiBinomial.Union2Tid(B0_5,B0_6)
B1_4 = TournoiBinomial.Union2Tid(B0_7,B0_8)
B2_2 = TournoiBinomial.Union2Tid(B1_3,B1_4)

B3_1 = TournoiBinomial.Union2Tid(B2_1,B2_2)
B3_1.plot()